In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': 'Jan_LSTM_noreg'
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [16]:
dfAll = pd.read_pickle("PklData/df_lstm_norm50.pkl")

lst = dfAll.userID.unique()
np.random.seed(1337)
np.random.shuffle(lst)
test_ids = lst[-5:]
train_ids = lst[:-5]
print(train_ids, test_ids)

[15  6 14  4  2  1 12 16  7 18  5 13  3] [11 17 10  8  9]


In [17]:
dfAll.userID.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

In [18]:
dfAll.TaskID = dfAll.TaskID % 17


In [19]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [20]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,50,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,50,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values

x_train = x_train / 255.0
x_test = x_test / 255.0

In [21]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique())
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [22]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allow_growth=True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.3

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
batch_size = 50
epochs = 5000
timesteps = 50
data_dim = (27,15)
l1v = 0.007
l2v = 0.014


tf.get_default_graph()
model = Sequential()

model.add(TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', 
                                 padding='same'), input_shape=(timesteps ,27, 15, 1)))
model.add(TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.50)))

model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', 
            padding='same', input_shape=(timesteps ,27, 15, 1))))
model.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.50)))

model.add(TimeDistributed(Flatten()))
#model.add(TimeDistributed(Dense(128)))
model.add(TimeDistributed(Dense(128)))
#model.add(TimeDistributed(Dense(32)))
model.add(LSTM(250, return_sequences=True, input_shape=(timesteps, data_dim)))
model.add(Dropout(0.25))
#model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, data_dim),kernel_regularizer=regularizers.l1_l2(0.001,0.01)))
#model.add(Dense(512))
model.add(LSTM(100, return_sequences=False, input_shape=(timesteps, data_dim)))
model.add(Dropout(0.25))
#model.add(Dense(256))
#model.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(0.20))
#model.add(Dense(64))
model.add(Dense(num_classes, activation='softmax'))

#optimizer = optimizers.Adagrad()
optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# Broadcast progress to the tensorboard.

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n" 
config += "l1: " + str(l1v) + "\n\n" + "l2: " + str(l2v) + "\n\n"

model.summary()
current_name = "KnuckleFinger_LSTM_Jan_noreg"
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/" + current_name + readable_timestamp
print(current_name + readable_timestamp)
logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0,
                            write_graph=True, write_images=True, update_freq = 'epoch')
storer = ModelCheckpoint("./ModelSnapshots/" + current_name + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                         save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=30, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)

history = model.fit(x_train, y_train_one_hot,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test_one_hot),
                        callbacks=[storer,logger,tg_callback, learning_rate_reduction])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_31 (TimeDis (None, 50, 27, 15, 64)    640       
_________________________________________________________________
time_distributed_32 (TimeDis (None, 50, 27, 15, 32)    18464     
_________________________________________________________________
time_distributed_33 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_34 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_35 (TimeDis (None, 50, 14, 8, 32)     9248      
_________________________________________________________________
time_distributed_36 (TimeDis (None, 50, 14, 8, 16)     4624      
_________________________________________________________________
time_distributed_37 (TimeDis (None, 50, 7, 4, 16)      0         
__________

7044/7044 [==============================] - 102s 14ms/step - loss: 0.1449 - acc: 0.9634 - val_loss: 0.4420 - val_acc: 0.8823
Epoch 43/5000
7044/7044 [==============================] - 101s 14ms/step - loss: 0.1446 - acc: 0.9602 - val_loss: 0.4542 - val_acc: 0.8795
Epoch 44/5000
7044/7044 [==============================] - 102s 14ms/step - loss: 0.1467 - acc: 0.9598 - val_loss: 0.4793 - val_acc: 0.8706
Epoch 45/5000
7044/7044 [==============================] - 102s 14ms/step - loss: 0.1405 - acc: 0.9628 - val_loss: 0.4568 - val_acc: 0.8795
Epoch 46/5000
7044/7044 [==============================] - 101s 14ms/step - loss: 0.1348 - acc: 0.9620 - val_loss: 0.4844 - val_acc: 0.8799
Epoch 47/5000
7044/7044 [==============================] - 101s 14ms/step - loss: 0.1401 - acc: 0.9615 - val_loss: 0.5010 - val_acc: 0.8697
Epoch 48/5000
7044/7044 [==============================] - 102s 14ms/step - loss: 0.1341 - acc: 0.9654 - val_loss: 0.4593 - val_acc: 0.8823
Epoch 49/5000
7044/7044 [=========

7044/7044 [==============================] - 100s 14ms/step - loss: 0.0606 - acc: 0.9832 - val_loss: 0.5336 - val_acc: 0.8855
Epoch 101/5000
7044/7044 [==============================] - 100s 14ms/step - loss: 0.0584 - acc: 0.9837 - val_loss: 0.5711 - val_acc: 0.8795
Epoch 102/5000
7044/7044 [==============================] - 100s 14ms/step - loss: 0.0501 - acc: 0.9871 - val_loss: 0.5659 - val_acc: 0.8776

Epoch 00102: ReduceLROnPlateau reducing learning rate to 9.02499959920533e-05.
Epoch 103/5000
7044/7044 [==============================] - 99s 14ms/step - loss: 0.0556 - acc: 0.9851 - val_loss: 0.5648 - val_acc: 0.8753
Epoch 104/5000
7044/7044 [==============================] - 100s 14ms/step - loss: 0.0529 - acc: 0.9851 - val_loss: 0.5302 - val_acc: 0.8869
Epoch 105/5000
7044/7044 [==============================] - 100s 14ms/step - loss: 0.0501 - acc: 0.9862 - val_loss: 0.6325 - val_acc: 0.8706
Epoch 106/5000
7044/7044 [==============================] - 100s 14ms/step - loss: 0.0471 

In [ ]:
print("end")

In [ ]:
model.save('lstm_28_02_19_inter_final.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_acc'], label="Test Accuracy")
plt.plot(history.history['acc'], label="Training Accuracy")
plt.legend()


In [ ]:

y_test_pred = model.predict(x_test, batch_size=30)
y_test_pred = np.argmax(y_test_pred, axis=1)
#model.predict(x_test)

print ('\n Summary of the precision, recall, F1 score for each class:')
print (sklearn.metrics.classification_report(y_test, y_test_pred))

print ('\n Confusion matrix: ')
print (sklearn.metrics.confusion_matrix(y_test, y_test_pred))